In [60]:
import os
from dotenv import load_dotenv

# Load the 'env' file
load_dotenv('env')

# Access your API key
api_key = os.getenv('API_KEY')

In [51]:
import requests
import json
from collections import defaultdict

# API endpoint
url = "https://comtradeapi.un.org/data/v1/get/C/M/HS"

# Parameters for the request
params = {
    "reporterCode": "566",  # Code for Nigeria
    "period": "202205",     # May 2022
    "partnerCode": "0",     # 0 for World (all partners)
    "cmdCode": "TOTAL",     # To get all HS codes
    "flowCode": "M,X"       # M for imports, X for exports
}

# Headers (you might need to replace 'your_subscription_key' with an actual key)
headers = {
    "Cache-Control": "no-cache",
    "Ocp-Apim-Subscription-Key": api_key
}

# Make the GET request
response = requests.get(url, params=params, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    
    # Dictionaries to store import and export values
    import_values = defaultdict(float)
    export_values = defaultdict(float)
    
    # Process the data
    for item in data.get('data', []):
        hs_code = item.get('cmdCode')[:2]  # Get the first 2 digits of the HS code
        trade_value = item.get('primaryValue', 0)
        
        if item.get('flowCode') == 'M':
            import_values[hs_code] += trade_value
        elif item.get('flowCode') == 'X':
            export_values[hs_code] += trade_value
    
    # Print the results
    print("Nigeria's Trade Data for May 2022 (2-digit HS classification)")
    print("\nImports:")
    for hs_code, value in import_values.items():
        print(f"HS {hs_code}: ${value:,.2f}")
    
    print("\nExports:")
    for hs_code, value in export_values.items():
        print(f"HS {hs_code}: ${value:,.2f}")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Nigeria's Trade Data for May 2022 (2-digit HS classification)

Imports:
HS TO: $4,941,153,768.25

Exports:
HS TO: $6,344,039,615.80


In [64]:
import requests
import json
from collections import defaultdict
from typing import List, Dict
from datetime import datetime, timedelta
import time

def get_comtrade_data(
    reporter_codes: List[str],
    start_year: int,
    end_year: int,
    flow_codes: str = "M,X",
    partner_code: str = "0",
    cmd_code: str = "TOTAL",
    api_key: str = "your_subscription_key",
    max_retries: int = 5,
    base_delay: float = 2.0
) -> Dict[str, Dict[str, Dict[str, float]]]:
    """
    Fetch trade data from the UN Comtrade API for multiple countries and years.
    Optimized to request larger chunks of data at once.

    Args:
    reporter_codes (List[str]): List of country codes to fetch data for.
    start_year (int): Start year for data collection.
    end_year (int): End year for data collection.
    flow_codes (str): 'M' for imports, 'X' for exports, or 'M,X' for both.
    partner_code (str): Partner country code. Default is '0' for all partners.
    cmd_code (str): Commodity code. Default is 'TOTAL' for all HS codes.
    api_key (str): API subscription key.
    max_retries (int): Maximum number of retries for a single request.
    base_delay (float): Base delay between requests in seconds.

    Returns:
    Dict[str, Dict[str, Dict[str, float]]]: Nested dictionary with trade data.
    """
    url = "https://comtradeapi.un.org/data/v1/get/C/M/HS"
    headers = {
        "Cache-Control": "no-cache",
        "Ocp-Apim-Subscription-Key": api_key
    }

    results = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(float))))

    def make_request_with_backoff(params):
        for attempt in range(max_retries):
            try:
                response = requests.get(url, params=params, headers=headers)
                response.raise_for_status()
                return response.json()
            except requests.exceptions.RequestException as e:
                if response.status_code == 429:  # Too Many Requests
                    wait_time = base_delay * (2 ** attempt)
                    print(f"Rate limit exceeded. Waiting for {wait_time:.2f} seconds before retrying...")
                    time.sleep(wait_time)
                else:
                    print(f"Error occurred: {e}")
                    if attempt == max_retries - 1:
                        print(f"Max retries reached. Skipping this request.")
                        return None
        return None

    for reporter_code in reporter_codes:
        for year in range(start_year, end_year + 1):
            params = {
                "reporterCode": reporter_code,
                "period": f"{year}01,{year}02,{year}03,{year}04,{year}05,{year}06,{year}07,{year}08,{year}09,{year}10,{year}11,{year}12",
                "partnerCode": partner_code,
                "cmdCode": cmd_code,
                "flowCode": flow_codes,
                "frequencyCode": "M"  # Monthly frequency
            }

            data = make_request_with_backoff(params)

            if data:
                for item in data.get('data', []):
                    period = item.get('period')
                    hs_code = item.get('cmdCode')
                    trade_value = item.get('primaryValue', 0)
                    flow_code = item.get('flowCode')
                    
                    results[reporter_code][period][hs_code][flow_code] += trade_value
            else:
                print(f"Failed to fetch data for {reporter_code} in {year}")

            # Add a delay between requests to avoid hitting rate limits
            time.sleep(base_delay)

    return results

def print_trade_data(trade_data: Dict[str, Dict[str, Dict[str, Dict[str, float]]]]):
    """
    Print the trade data in a readable format.

    Args:
    trade_data (Dict[str, Dict[str, Dict[str, Dict[str, float]]]]): The trade data to print.
    """
    for country, years in trade_data.items():
        print(f"\nTrade Data for Country Code: {country}")
        for period, hs_codes in years.items():
            print(f"\n  Period: {period}")
            for hs_code, flows in hs_codes.items():
                print(f"    HS Code {hs_code}:")
                for flow, value in flows.items():
                    flow_name = "Imports" if flow == "M" else "Exports"
                    print(f"      {flow_name}: ${value:,.2f}")

# Example usage
if __name__ == "__main__":
    reporter_codes = ["566", "826"]  # Nigeria and United Kingdom
    start_year = 2020
    end_year = 2023
    flow_codes = "M,X"     # Both imports and exports
    api_key = api_key

    trade_data = get_comtrade_data(
        reporter_codes=reporter_codes,
        start_year=start_year,
        end_year=end_year,
        flow_codes=flow_codes,
        api_key=api_key,
        max_retries=5,
        base_delay=2.0
    )

    print_trade_data(trade_data)

Rate limit exceeded. Waiting for 2.00 seconds before retrying...

Trade Data for Country Code: 566

  Period: 202001
    HS Code TOTAL:
      Imports: $10,354,064,699.11
      Exports: $11,119,931,039.50

  Period: 202002
    HS Code TOTAL:
      Exports: $9,374,809,018.81
      Imports: $8,677,478,118.69

  Period: 202003
    HS Code TOTAL:
      Exports: $5,904,383,593.24
      Imports: $9,299,683,146.92

  Period: 202004
    HS Code TOTAL:
      Exports: $3,801,426,948.38
      Imports: $6,195,574,789.17

  Period: 202005
    HS Code TOTAL:
      Exports: $4,169,909,720.46
      Imports: $7,492,809,977.67

  Period: 202006
    HS Code TOTAL:
      Imports: $9,301,591,513.98
      Exports: $4,355,259,800.65

  Period: 202007
    HS Code TOTAL:
      Imports: $9,618,243,923.66
      Exports: $5,858,525,519.89

  Period: 202008
    HS Code TOTAL:
      Exports: $5,207,928,084.83
      Imports: $9,672,141,736.11

  Period: 202009
    HS Code TOTAL:
      Imports: $9,062,793,151.38
     

defaultdict(<function __main__.get_comtrade_data.<locals>.<lambda>()>,
            {'566': defaultdict(<function __main__.get_comtrade_data.<locals>.<lambda>.<locals>.<lambda>()>,
                         {'202001': defaultdict(<function __main__.get_comtrade_data.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>()>,
                                      {'TOTAL': defaultdict(float,
                                                   {'M': 10354064699.109001,
                                                    'X': 11119931039.503})}),
                          '202002': defaultdict(<function __main__.get_comtrade_data.<locals>.<lambda>.<locals>.<lambda>.<locals>.<lambda>()>,
                                      {'TOTAL': defaultdict(float,
                                                   {'X': 9374809018.81,
                                                    'M': 8677478118.686003})}),
                          '202003': defaultdict(<function __main__.get_comtrade_data.<locals>